# Notebook to summarize shurf model results

In [1]:
import pandas as pd
import numpy as np
import statistics

import xarray as xr
import CRPS.CRPS as pscore
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


In [2]:
# Paths
path = '/Users/havardhegre1/views-platform/views-models/models/'
model_list = [
    {'name': 'wild_rose',
    'stub': 'predictions_validation_20251008_225829_',
    'weight': 500}, 
    {'name': 'wuthering_heights',
    'stub': 'predictions_validation_20251009_111143_',
    'weight': 500},
    {'name': 'lovely_creature',
    'stub': 'predictions_validation_20251008_090841_',
    'weight': 500},
    {'name': 'purple_haze',
    'stub': 'predictions_validation_20251010_090625_',
    'weight': 500},
    {'name': 'great_gig',
    'stub': 'predictions_validation_20251011_075859_',
    'weight': 500},
    {'name': 'fourtieth_symphony',
    'stub': 'predictions_validation_20251013_090203_',
    'weight': 500},
#    {'name': 'cheap_thrills',
#    'stub': 'predictions_validation_20250411_080754_',
#    'weight': 100},
]
sequences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']

#number_of_sequences = len(sequences) # 12
number_of_sequences = 1 # For testing

number_of_models = len(model_list) # 5
total_weight = sum([model['weight'] for model in model_list]) # 1600
outcome = 'lr_sb_best'
predcol = 'pred_' + outcome

for model in model_list:
    filename = f"{path}{model['name']}/data/generated/{model['stub']}"
#    print(model['name'], model['stub'], filename)
    model['seq_data'] = []
    for s in sequences[0:number_of_sequences]:
        df = pd.read_parquet(filename + f"{s}.parquet")
        try:
            df.rename(columns={'pred_ln_sb_best': predcol}, inplace=True)
        except:
            pass
        try:
            df.rename(columns={'pred_sb_best': predcol}, inplace=True)
        except:
            pass
        df_dict = {
            'first_month': df.index[0][0],
            'last_month': df.index[-1][0],
            'samples': len(df.loc[df.index[0][0], 1].to_list()[0]),
            'data': df
        }
        print(model['name'], s, df_dict['first_month'], df_dict['last_month'], df_dict['samples'])
        model['seq_data'].append(df_dict)


wild_rose 00 493 528 500
wuthering_heights 00 493 528 500
lovely_creature 00 493 528 5
purple_haze 00 493 528 500
great_gig 00 493 528 500
fourtieth_symphony 00 493 528 500


In [3]:
model_list[-1]['seq_data'][-1]['data']



pred_lr_sb_best
month_id country_id                                                   
493      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         2           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         3           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         4           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         5           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
...                                                                ...
528      242         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         245         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
         246         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

[6876 rows x 1 columns]

In [4]:
config_hyperparameters = {
    'wild_rose': 'hyperparameters_20250408_164044.json',
    'wuthering_heights': 'hyperparameters_20250410_074521.json',
    'lovely_creature': 'hyperparameters_20250408_103715.json',
    'fourtieth_symphony': 'hyperparameters_20250408_213359.json',
    'cheap_thrills': 'hyperparameters_20250411_080754.json',
}
config_hyperparameters

{'wild_rose': 'hyperparameters_20250408_164044.json',
 'wuthering_heights': 'hyperparameters_20250410_074521.json',
 'lovely_creature': 'hyperparameters_20250408_103715.json',
 'fourtieth_symphony': 'hyperparameters_20250408_213359.json',
 'cheap_thrills': 'hyperparameters_20250411_080754.json'}

In [5]:
# Extract model characteristics
for model in model_list:
    filename = f"{path}{model['name']}/configs/config_hyperparameters.py"
    with open(filename, 'r') as file:
        lines = file.readlines()
    print(model['name'])
#    print(lines[12:])
    filename = f"{path}{model['name']}/configs/config_meta.py"
    with open(filename, 'r') as file:
        lines = file.readlines()
#    print(lines[12:])   
    hp_dict = {
        'Name': lines[10].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", ""),
        'Algorithm': lines[11].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", ""),
        'Target': lines[12].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", "").replace("[", "").replace("]", ""),
        'Model regression': lines[15].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", ""),
        'Model classification': lines[16].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", ""),
        'Queryset': lines[18].split(':')[1].strip().replace(",", "").replace("'", "").replace("\"", ""),
        
    }
    print(hp_dict)

wild_rose
{'Name': 'wild_rose', 'Algorithm': 'ShurfModel', 'Target': 'lr_sb_best', 'Model regression': 'XGBRegressor', 'Model classification': 'XGBClassifier', 'Queryset': 'uncertainty_conflict_nolog'}
wuthering_heights
{'Name': 'wuthering_heights', 'Algorithm': 'ShurfModel', 'Target': 'lr_sb_best', 'Model regression': 'XGBRegressor', 'Model classification': 'XGBClassifier', 'Queryset': 'uncertainty_deep_conflict_nolog'}
lovely_creature
{'Name': 'lovely_creature', 'Algorithm': 'ShurfModel', 'Target': 'lr_sb_best', 'Model regression': 'XGBRegressor', 'Model classification': 'XGBClassifier', 'Queryset': 'uncertainty_broad_nolog'}
purple_haze
{'Name': 'purple_haze', 'Algorithm': 'ShurfModel', 'Target': 'lr_sb_best', 'Model regression': 'XGBRegressor', 'Model classification': 'XGBClassifier', 'Queryset': 'uncertainty_narrow_nolog'}
great_gig
{'Name': 'lovely_creature', 'Algorithm': 'ShurfModel', 'Target': 'lr_sb_best', 'Model regression': 'XGBRegressor', 'Model classification': 'XGBClassif

In [6]:
def check_setup(m, s, first_month=447, country=246, with_actuals=False):
    print(model_list[m]['name'])
    print('data:',model_list[m]['seq_data'][s]['data'].head(5))
    print('data.loc[first_month,country]:',model_list[m]['seq_data'][s]['data'].loc[first_month,country].head(5))
    if with_actuals:
        print('df_w_actuals:',model_list[m]['seq_data'][s]['df_w_actuals'].head(5))
        print('df_w_actuals.loc[first_month,country]:',model_list[m]['seq_data'][s]['df_w_actuals'].loc[first_month,country].head(5))



# Smoothing predictions through a sampling process

In [7]:
def smoothing_matrix(steps, option = 0, samples = 1000, verbose=False):
    # Create a smoothing matrix
    smoothing_matrix = np.zeros((steps, steps))
    if option == 0:
        smoothing_matrix[0, 0:4] = [550, 250, 150, 50]
        smoothing_matrix[1, 0:5] = [225, 475, 175, 75, 50]
        smoothing_matrix[2, 0:6] = [50, 200, 400, 200, 100, 50]
        for i in range(3, steps-3):
            smoothing_matrix[i, i-3:i+4] = [50, 100, 200, 300, 200, 100, 50]
        smoothing_matrix[steps-3, steps-7:steps] = [50, 100, 200, 300, 200, 100, 50]
        smoothing_matrix[steps-2, steps-6:steps] = [50, 100, 150, 200, 300, 200]
        smoothing_matrix[steps-1, steps-5:steps] = [50, 100, 150, 250, 450]
        
    if option == 1:
        smoothing_matrix[0, 0:5] = [600, 200, 100, 75, 25]
        smoothing_matrix[1, 0:6] = [200, 350, 175, 125, 100, 50]
        smoothing_matrix[2, 0:8] = [100, 200, 325, 125, 100, 75, 50, 25]
        smoothing_matrix[3, 0:8] = [50, 100, 175, 325, 150, 100, 75, 25]
        for i in range(4, steps-4):
            smoothing_matrix[i, i-4:i+5] = [25, 75, 100, 150, 300, 150, 100, 75, 25]
        smoothing_matrix[steps-4, steps-9:steps] = [25, 75, 100, 150, 275, 150, 125, 75, 25]
        smoothing_matrix[steps-3, steps-8:steps] = [25, 50, 75, 100, 300, 200, 150, 100]
        smoothing_matrix[steps-2, steps-7:steps] = [0, 50, 75, 100, 175, 350, 250]
        smoothing_matrix[steps-1, steps-6:steps] = [0, 25, 75, 125, 275, 500]
    # Check rows and columns sum to 1000
    if verbose:
        for i in range(steps):
            if np.sum(smoothing_matrix[i, :]) != 1000:
                print('Horizontal check:', i, np.sum(smoothing_matrix[i, :]))
            if np.sum(smoothing_matrix[:, i]) != 1000:
                print('Vertical check:', i, np.sum(smoothing_matrix[:, i]))
    # Scale to samples
    smoothing_matrix = smoothing_matrix * (samples / 1000)
    return smoothing_matrix.astype(int)
    
sm = smoothing_matrix(36, option=1, samples = 500, verbose=True)
sm

Vertical check: 0 975.0
Vertical check: 1 950.0
Vertical check: 2 975.0
Vertical check: 4 1025.0
Vertical check: 5 1025.0
Vertical check: 6 1025.0
Vertical check: 7 1025.0
Vertical check: 27 1025.0
Vertical check: 28 1075.0
Vertical check: 29 1050.0
Vertical check: 30 1075.0
Vertical check: 31 1125.0
Vertical check: 32 975.0
Vertical check: 33 825.0
Vertical check: 34 950.0
Vertical check: 35 900.0


array([[300, 100,  50, ...,   0,   0,   0],
       [100, 175,  87, ...,   0,   0,   0],
       [ 50, 100, 162, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 100,  75,  50],
       [  0,   0,   0, ...,  87, 175, 125],
       [  0,   0,   0, ...,  62, 137, 250]])

In [8]:

def smooth_predictions(m, s, smoothing_matrix):
    ''' Smooth predictions for model m and sequence s by sampling from adjacent steps according to the smoothing matrix '''
    df = model_list[m]['seq_data'][s]['data']
    df['smoothed_predictions'] = None
#    print(df.head(5))
    to_step = 0
    for to_month in df.index.get_level_values(0).unique():
#        print('smoothing month', to_month)
        for country in df.loc[to_month].index:
#            print('smoothing country', country)
            sampled_predictions = []
            for from_step in range(36):
                from_month = to_month - to_step + from_step
#                print('to_month:', to_month, 'to_step:', to_step, 'from_step:', from_step, 'from_month:', from_month, 'weight:', smoothing_matrix[to_step, from_step], df.loc[from_month, country])
                sampled_predictions = sampled_predictions + np.random.choice(df.loc[from_month, country][predcol], smoothing_matrix[to_step, from_step], replace=True).tolist()
            df['smoothed_predictions'].loc[to_month, country] = sampled_predictions
        to_step += 1
    return df      

# Smooth predictions for model m and sequence s
#for model in model_list[0:1]:
#    for s in range(1):
for model in model_list[0:number_of_models]:
    for s in range(number_of_sequences):
        print('Smoothing model', model['name'], 'sequence', s)
        sm = smoothing_matrix(36, option=1)
        model['seq_data'][s]['data'] = smooth_predictions(model_list.index(model), s, sm)
#        print(model['seq_data'][s]['data'].head(5))

Smoothing model wild_rose sequence 0
Smoothing model wuthering_heights sequence 0
Smoothing model lovely_creature sequence 0
Smoothing model purple_haze sequence 0
Smoothing model great_gig sequence 0
Smoothing model fourtieth_symphony sequence 0


In [9]:
# First and last months
first_month = model_list[-1]['seq_data'][0]['first_month']
last_month = model_list[-1]['seq_data'][-1]['last_month']
print(first_month, last_month, last_month - first_month)

493 528 35


In [10]:
# checking results for smoothing the predictions
m = 0
s = number_of_sequences - 1
month = first_month + s
country = 245

print(model_list[m]['seq_data'][s]['data'].tail(5))
print('originals')
print('length',len(model_list[m]['seq_data'][s]['data'].loc[month, country][predcol]))
print('mean',np.mean(model_list[m]['seq_data'][s]['data'].loc[month, country][predcol]))
print('sd',statistics.stdev(model_list[m]['seq_data'][s]['data'].loc[month, country][predcol]))
print('smoothed')
print('length',len(model_list[m]['seq_data'][s]['data'].loc[month, country]['smoothed_predictions']))
print('mean',np.mean(model_list[m]['seq_data'][s]['data'].loc[month, country]['smoothed_predictions']))
print('sd',statistics.stdev(model_list[m]['seq_data'][s]['data'].loc[month, country]['smoothed_predictions']))

                                                       pred_lr_sb_best  \
month_id country_id                                                      
528      242         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         245         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         246         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                  smoothed_predictions  
month_id country_id                                                     
528      242         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         245         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         246         [232.0, 20.0, 158.0, 3

# Ensemble

In [11]:
model_list[1]['seq_data'][0]['data']

pred_lr_sb_best  \
month_id country_id                                                      
493      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         2           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         3           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         4           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         5           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                                ...   
528      242         [7.0, 123.0, 25.0, 40.0, 36.0, 19.0, 22.0, 25....   
         243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         245         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
         246         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                  smoothed_predictions  
month_id country_id                                                     
493      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         2           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         3           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         4           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         5           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
...                                                                ...  
528      242         [110.0, 29.0, 15.0, 128.0, 19.0, 18.0, 13.0, 8...  
         243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         245         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
         246         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  

[6876 rows x 2 columns]

In [12]:
# Function to compute ensemble predictions

def compute_ensembles(predcol, model_list, ensemble_name, number_of_sequences, smoothed=False):
    
    if smoothed:
        predictions = 'smoothed_predictions'
    else:
        predictions = predcol
        
    print('Computing ensemble for', predictions, ensemble_name)

    ensemble_dict = {
        'stub': '',
        'weight': 1000,        
    }
    ensemble_data = []

#    for s in range(len(sequences)):
    for s in range(number_of_sequences):
        print('Sequence:', s)
        m = 0
        for model in model_list[0:number_of_models]:
            df_dict = {
                'first_month': model['seq_data'][s]['first_month'],
                'last_month': model['seq_data'][s]['last_month'],
                'samples': model['weight'],
            }
#            print(model['seq_data'][s]['data'].tail(5))
            df = pd.DataFrame(model['seq_data'][s]['data'][predictions]).copy()
#            print('Orginal:', len(df.loc[first_month+12,245][predictions]), np.mean(df.loc[first_month+12,245][predictions]),df.tail(1))
#            print(predictions)
            df = df.explode(predictions, ignore_index=False)
#            print('Exploded:',df.tail(3))
            df.reset_index(inplace=True)
#            print(df.tail(3))
            df['draw'] = df.groupby(['month_id','country_id',predictions]).cumcount()
            df.set_index(['month_id','country_id','draw'], inplace=True)
            df.sort_index(inplace=True)
            df = df.loc[:, :, 0:model['weight']-1] # keep only as many draws as weight
            df.reset_index(inplace=True)
            df_imploded = df.groupby(['month_id', 'country_id'])[predictions].apply(lambda x: x.tolist()).reset_index()
            df_imploded.set_index(['month_id', 'country_id'], inplace=True)
            df = df_imploded
#            print('Exploded:', len(df.loc[480,245][predictions]), np.mean(df.loc[480,245][predictions]),df.tail(1))

#            print(m, model['name'], model['weight'], s, model['seq_data'][s]['first_month'])
            
            if m == 0:
                ensemble_df = df
            else:
                ensemble_df[predictions] = ensemble_df[predictions] + df[predictions]
#            print('Ensemble:', len(ensemble_df.loc[480,245][predictions]), np.mean(ensemble_df.loc[480,245][predictions]),ensemble_df.tail(1))
            
            m+=1
    #    print(ensemble_df.loc[447,246].head(5))
        if predictions == 'smoothed_predictions':
            ensemble_df.rename(columns={'smoothed_predictions': predcol}, inplace=True)
        df_dict['data'] = ensemble_df
        ensemble_data.append(df_dict)
            

        ensemble_dict = {
            'name': ensemble_name,
            'stub': '',
            'weight': 1000,
            'seq_data': ensemble_data
        }
        print(ensemble_dict['name'], len(ensemble_dict['seq_data']))
        return ensemble_dict

ensemble_dict = compute_ensembles(predcol, model_list[0:6], 'ensemble_raw', number_of_sequences, smoothed=False)
model_list.append(ensemble_dict)

ensemble_dict = compute_ensembles(predcol, model_list[0:6], 'ensemble_smoothed', number_of_sequences, smoothed=True)
model_list.append(ensemble_dict)

for model_ablations in range(0, number_of_models):
    print('Ablation removing model', model_ablations, model_list[model_ablations]['name'])
    ablation_model_list = [model_list[i] for i in range(number_of_models) if i != model_ablations]
    ensemble_dict = compute_ensembles(predcol, ablation_model_list, f'ensemble_smoothed_no_{model_list[model_ablations]["name"]}', number_of_sequences, smoothed=True)
    model_list.append(ensemble_dict)

Computing ensemble for pred_lr_sb_best ensemble_raw
Sequence: 0
ensemble_raw 1
Computing ensemble for smoothed_predictions ensemble_smoothed
Sequence: 0
ensemble_smoothed 1
Ablation removing model 0 wild_rose
Computing ensemble for smoothed_predictions ensemble_smoothed_no_wild_rose
Sequence: 0
ensemble_smoothed_no_wild_rose 1
Ablation removing model 1 wuthering_heights
Computing ensemble for smoothed_predictions ensemble_smoothed_no_wuthering_heights
Sequence: 0
ensemble_smoothed_no_wuthering_heights 1
Ablation removing model 2 lovely_creature
Computing ensemble for smoothed_predictions ensemble_smoothed_no_lovely_creature
Sequence: 0
ensemble_smoothed_no_lovely_creature 1
Ablation removing model 3 purple_haze
Computing ensemble for smoothed_predictions ensemble_smoothed_no_purple_haze
Sequence: 0
ensemble_smoothed_no_purple_haze 1
Ablation removing model 4 great_gig
Computing ensemble for smoothed_predictions ensemble_smoothed_no_great_gig
Sequence: 0
ensemble_smoothed_no_great_gig 1

In [13]:
for model in model_list:
    print(model['name'])

wild_rose
wuthering_heights
lovely_creature
purple_haze
great_gig
fourtieth_symphony
ensemble_raw
ensemble_smoothed
ensemble_smoothed_no_wild_rose
ensemble_smoothed_no_wuthering_heights
ensemble_smoothed_no_lovely_creature
ensemble_smoothed_no_purple_haze
ensemble_smoothed_no_great_gig
ensemble_smoothed_no_fourtieth_symphony


# Extract actuals

In [16]:
# Running from PRIO:
!viewser config set REMOTE_URL https://viewser.viewsforecasting.org
# Running from Uppsala:

REMOTE_URL: https://viewser.viewsforecasting.org


In [17]:

from viewser.operations import fetch
from viewser import Queryset, Column
def countrydata():
    qs = (Queryset("country_data_for_evaluation", "country_month")
    .with_column(Column("country_name",from_loa="country",from_column="name"))
    .with_column(Column("isonum",from_loa="country",from_column="isonum"))
    .with_column(Column("isoab",from_loa="country",from_column="isoab"))
    .with_column(Column(outcome, from_loa='country_month', from_column='ged_sb_best_sum_nokgi')
        .transform.missing.fill()
        .transform.missing.replace_na()
        )
    )
    data = qs.publish().fetch()
    return(data)


historical_actuals_12_df = countrydata().loc[first_month-12-1:last_month-36, :]
print(historical_actuals_12_df)
historical_actuals_36_df = countrydata().loc[first_month-36-1:last_month-36, :]
print(historical_actuals_36_df)

actuals_df = countrydata().loc[first_month:last_month, :]
print(actuals_df)

Queryset country_data_for_evaluation read successfully 
                            country_name  isonum isoab  lr_sb_best
month_id country_id                                               
480      1                        Guyana     328   GUY           0
         2                      Suriname     740   SUR           0
         3           Trinidad and Tobago     780   TTO           0
         4                     Venezuela     862   VEN           0
         5                         Samoa     882   WSM           0
...                                  ...     ...   ...         ...
492      242                    Tanzania     834   TZA           0
         243                     Morocco     504   MAR           0
         244                  Mauritania     478   MRT           0
         245                       Sudan     729   SDN           0
         246                 South Sudan     728   SSD           0

[2483 rows x 4 columns]
Queryset country_data_for_evaluation read succes

In [ ]:
# Figure for Schincariol defense

schincariol_df = countrydata()
schincariol_df.head()


In [18]:
# Plot fatalities for three countries of interest:

selected_countries = ['Syria', 'Ukraine', 'Ethiopia']
for country in selected_countries:

    country_df = schincariol_df.loc[schincariol_df['country_name'] == country, :]
    country_df = country_df.loc[380:540]
    country_df.reset_index(inplace=True)
    country_df['year_2023'] = country_df['month_id'].apply(lambda x: 1 if x >= 517 and x <= 528 else 0)*10000
    country_df.head()
    country_df.plot(x='month_id', y=[outcome,'year_2023'], title=f'Reported fatalities in {country}')
    


NameError: name 'schincariol_df' is not defined

# Onset risk classification and conflictology

In [19]:
historical_actuals_12_df.head()

country_name  isonum isoab  lr_sb_best
month_id country_id                                               
480      1                        Guyana     328   GUY           0
         2                      Suriname     740   SUR           0
         3           Trinidad and Tobago     780   TTO           0
         4                     Venezuela     862   VEN           0
         5                         Samoa     882   WSM           0

In [20]:
def at_risk_of_onset(historical_actuals):
    ''' Classify country-months as at risk of onset (1) or not (0) based on historical actuals for the last 12 months'''
    if len(historical_actuals) != 12 and len(historical_actuals) != 36:
        print('Error: historical_actuals must have length 12 or 36')
        return None
    # Check if there was any conflict in the last 12 months
    any_conflict = any([x > 0 for x in historical_actuals])
    # Check if there was any onset in the last 12 months
#    any_onset = any([historical_actuals[i] == 0 and historical_actuals[i+1] > 0 for i in range(11)])
    if not any_conflict:
        return 1
    else:
        return 0


    
ha0 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	
print(at_risk_of_onset(ha0))
ha1 = [0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0]	
print(at_risk_of_onset(ha1))

1
0


In [21]:
first_month, first_month+11, first_month+11+35


(493, 504, 539)

In [22]:

def create_conflictologies(historical_actuals_df, number_of_sequences, first_month, months=12):
    ''' Create conflictology and onset risk classification for each sequence'''         

    conflictology_list = []
    for s in range(number_of_sequences):
        last_month_with_data = first_month + s - 1
#        print('Sequence', s, 'last month with data:', last_month_with_data)
        last_months_df = pd.DataFrame(historical_actuals_df.loc[last_month_with_data-(months-1):last_month_with_data, :])
        conflictology_stem = last_months_df.groupby(['country_id'], group_keys=True)[outcome].apply(lambda x: x.tolist()).reset_index()
        conflictology_stem.rename(columns={outcome: 'conflictology'}, inplace=True)
        conflictology_stem.set_index(['country_id'], inplace=True)
        conflictology_stem['at_risk_of_onset'] = conflictology_stem['conflictology'].apply(at_risk_of_onset)

        dict = {
            's': s,
            'last_month_with_data': last_month_with_data,
            'data': conflictology_stem,
            'months': months
        }
        conflictology_list.append(dict)

    return conflictology_list
#print(conflictology_list[3]['data'].head(25))   


In [23]:
model_list[0]

{'name': 'wild_rose',
 'stub': 'predictions_validation_20251008_225829_',
 'weight': 500,
 'seq_data': [{'first_month': 493,
   'last_month': 528,
   'samples': 500,
   'data':                                                        pred_lr_sb_best  \
   month_id country_id                                                      
   493      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            2           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            3           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            4           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            5           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
   ...                                                                ...   
   528      242         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [24]:
# Add conflictology as model in the model list
def conflictology(months, conflictology_list, over_sampling = 1):
    ''' Create conflictology model with given number of months and weight'''

    conflictology_model = {
        'name': f'conflictology_{months}_months',
        'stub': '',
        'weight': months,
        'seq_data': []
    }
    for s in range(number_of_sequences):
        dict = {
            'first_month': conflictology_list[s]['last_month_with_data'] - (months - 1),
            'last_month': conflictology_list[s]['last_month_with_data'],
            'samples': months,
        }
        df_c = conflictology_list[s]['data'].copy()
        df_c['month_id'] = first_month + s
        df_c.reset_index(inplace=True)  
        df_c.set_index(['month_id','country_id'], inplace=True) 
        for oversample in range(over_sampling - 1):   
            for month in range(first_month + s, first_month + s + 36 ):
                df_add = conflictology_list[s]['data'].copy()
                df_add['month_id'] = month
                df_add.reset_index(inplace=True)
                df_add.set_index(['month_id','country_id'], inplace=True)    
                df_c = pd.concat([df_c, df_add])
        dict['data'] = df_c
        
        dict['data'].rename(columns={'conflictology': predcol}, inplace=True)
        dict['data'].drop(columns=['at_risk_of_onset'], inplace=True)
        conflictology_model['seq_data'].append(dict)
    return conflictology_model    

# 12 months conflictology

conflictology_list = create_conflictologies(historical_actuals_12_df, number_of_sequences, first_month, 12)
s = number_of_sequences - 1
#print(conflictology_list[s])
conflictology_12_months = conflictology(12, conflictology_list, over_sampling = 40)
model_list.append(conflictology_12_months)


# 36 months conflictology

conflictology_list = create_conflictologies(historical_actuals_36_df, number_of_sequences, first_month, 36)
s = number_of_sequences - 1
#print(conflictology_list[s])
conflictology_36_months = conflictology(36, conflictology_list)
model_list.append(conflictology_36_months)

In [25]:
model_list[-1]['seq_data'][-1]['data']

pred_lr_sb_best
month_id country_id                                                   
493      1           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         2           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         3           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         4           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         5           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
...                                                                ...
         242         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         243         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         244         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
         245         [4, 2, 31, 15, 43, 30, 18, 3, 50, 4, 45, 9, 5,...
         246         [31, 50, 15, 110, 174, 73, 11, 10, 25, 5, 0, 3...

[191 rows x 1 columns]

In [26]:
# Add conflictology models to ensemble
def merge_ensembles(ensemble_1, ensemble_2, number_of_sequences, col_1, col_2, name='Merged_Ensemble'):
    ''' Merge two ensemble models by averaging their predictions '''
    merged_ensemble = {
        'name': name,
        'stub': '',
        'weight': 1000,
        'seq_data': []
    }
    
    print('Ensembles to merge: ', ensemble_1['name'], 'and', ensemble_2['name'])
    for s in range(number_of_sequences):
        df1 = ensemble_1['seq_data'][s]['data']
        df2 = ensemble_2['seq_data'][s]['data']
        merged_df = df1.explode(col_1, ignore_index=False).copy()
        merged_df.reset_index(inplace=True)
#        print(merged_df.tail(5))
        df2_exploded = df2.explode(col_2, ignore_index=False).copy()
        df2_exploded.reset_index(inplace=True)
        merged_df = pd.concat([merged_df, df2_exploded], axis=0)
#        print(merged_df.tail(5))
        merged_df['draw'] = merged_df.groupby(['month_id','country_id',col_1]).cumcount()
        merged_df.set_index(['month_id','country_id','draw'], inplace=True)
        merged_df.sort_index(inplace=True)
        merged_df.reset_index(inplace=True)
        merged_df_imploded = merged_df.groupby(['month_id', 'country_id'])[col_1].apply(lambda x: x.tolist()).reset_index()
        merged_df_imploded.set_index(['month_id', 'country_id'], inplace=True)
        
#        merged_df[predcol] = [(np.array(x) + np.array(y)) / 2 for x, y in zip(df1[predcol], df2[predcol])]
        df_dict = {
            'first_month': df1.index.get_level_values(0).min(),
            'last_month': df1.index.get_level_values(0).max(),
            'samples': 1000,
            'data': merged_df_imploded
        }
        merged_ensemble['seq_data'].append(df_dict)
    return merged_ensemble

merged_ensemble = merge_ensembles(model_list[-3], model_list[-2], number_of_sequences, predcol, predcol, name='Ensemble_with_conflictology')
merged_ensemble = merge_ensembles(merged_ensemble, model_list[-1], number_of_sequences, predcol, predcol, name='Ensemble_with_conflictology')
merged_ensemble
model_list.append(merged_ensemble)

Ensembles to merge:  ensemble_smoothed_no_fourtieth_symphony and conflictology_12_months
Ensembles to merge:  Ensemble_with_conflictology and conflictology_36_months


# Evaluation

In [27]:

def crps(predictions, actual):
    crps,fcrps,acrps =  pscore(predictions, actual).compute()
    return crps

def winkler_interval_score(y_true, y_lower, y_upper, alpha):
    """
    Calculates the Winkler Interval Score for a single observation.

    Args:
        y_true (float): The true observed value.
        y_lower (float): The lower bound of the prediction interval.
        y_upper (float): The upper bound of the prediction interval.
        alpha (float): 1 - coverage level (e.g., 0.1 for 90% coverage).

    Returns:
        float: The Winkler Interval Score.
    """
    if y_true < y_lower:
        return (y_upper - y_lower) + (2 / alpha) * (y_lower - y_true)
    elif y_true > y_upper:
        return (y_upper - y_lower) + (2 / alpha) * (y_true - y_upper)
    else:
        return (y_upper - y_lower)
    

def brier_score(predictions, actual, threshold):
    """
    Calculate the Brier Score for binary events.

    Args:
        predictions (list or np.array): Predicted probabilities for the event.
        actual (list or np.array): Actual binary outcomes (0 or 1).
        threshold (float): Probability threshold to classify an event as 1.

    Returns:
        float: The Brier Score.
    """
    predictions = np.array(predictions)
    actual_above_threshold = (actual >= threshold)
    # Apply threshold to predictions
    
    binary_predictions = (predictions >= threshold).sum() / len(predictions)
    # Calculate Brier Score
    return (binary_predictions - actual_above_threshold) ** 2


def categorize_fatalities(predlist):
    bins = [0,1,10,100,1000,10000] # Define bin edges
    counts, _ = np.histogram(predlist, bins=bins)
    proportions = counts / counts.sum()
    mode = statistics.mode(predlist)
    # Create labels for the bins
    labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]
    #return counts.tolist(), proportions.tolist(), len(predlist), mode, labels
    return mode, proportions.tolist()


In [28]:
actuals_df.tail()

country_name  isonum isoab  lr_sb_best
month_id country_id                                       
528      242            Tanzania     834   TZA           0
         243             Morocco     504   MAR           4
         244          Mauritania     478   MRT           0
         245               Sudan     729   SDN         364
         246         South Sudan     728   SSD           0

In [57]:
# Data for ensemble and all models

bands = [80, 90, 95, 98]
percentiles_list = []
for b in bands:
    p_low = (100 - b) / 2
    p_high = 100 - p_low
    percentiles_list.append(p_low)
    percentiles_list.append(p_high)
    
for model in model_list:
    print(model['name'])
    for s in range(number_of_sequences):
        print('Sequence:', s)
        df = model['seq_data'][s]['data']
        # merge with actuals    
        model['seq_data'][s]['df_w_actuals'] = df.merge(actuals_df, left_index=True, right_on=['month_id', 'country_id'], how='left')
        # Include at risk of onset indicator
        model['seq_data'][s]['df_w_actuals'] = model['seq_data'][s]['df_w_actuals'].merge(conflictology_list[s]['data'][['at_risk_of_onset']], left_on=['country_id'], right_index=True, how='left')
        
        # Compute crps
        model['seq_data'][s]['df_w_actuals']['crps'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: crps(row[predcol], row[outcome]), axis=1)
        # Compute crps for risk of onset only
        model['seq_data'][s]['df_w_actuals']['crps_onset'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: crps(row[predcol], row[outcome]) if row['at_risk_of_onset'] else np.nan, axis=1)
        # Compute percentiles and coverage
        model['seq_data'][s]['df_w_actuals']['percentiles'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: np.percentile(row[predcol], percentiles_list), axis=1)
        # Compute mean prediction
        model['seq_data'][s]['df_w_actuals']['mean_pred'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: np.mean(row[predcol]), axis=1)
        i = 0
        # Prediction intervals, coverage and Winkler score
        for b in bands:
            col = f'lower_{str(b)}'
            model['seq_data'][s]['df_w_actuals'][col] =model['seq_data'][s]['df_w_actuals'].apply(lambda row: np.percentile(row[predcol], percentiles_list[i]), axis=1)
            i = i + 1
            col = f'higher_{str(b)}'
            model['seq_data'][s]['df_w_actuals'][col] =model['seq_data'][s]['df_w_actuals'].apply(lambda row: np.percentile(row[predcol], percentiles_list[i]), axis=1)
            i = i + 1
            model['seq_data'][s]['df_w_actuals'][f'coverage_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: 1 if (row[outcome] >= row[f'lower_{str(b)}']) & (row[outcome] <= row[f'higher_{str(b)}']) else 0, axis=1)
            model['seq_data'][s]['df_w_actuals'][f'winkler_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: winkler_interval_score(row[outcome], row[f'lower_{str(b)}'], row[f'higher_{str(b)}'], (1 - b/100)), axis=1)
            model['seq_data'][s]['df_w_actuals']['mse'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: mean_squared_error([row[outcome]], [np.mean(row[predcol])]), axis=1)
        # Categorize fatalities,  thresholds 1, 10, 100, 1000, 10000, and add mode and proportions to dataframe
        model['seq_data'][s]['df_w_actuals']['fatality_categories'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: categorize_fatalities(row[predcol])[1], axis=1)
        model['seq_data'][s]['df_w_actuals']['categories_mode'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: categorize_fatalities(row[predcol])[0], axis=1)
        # Brier scores for thresholds 1, 10, 100, 1000, 10000
        for threshold in [1, 10, 100, 1000, 10000]:
            model['seq_data'][s]['df_w_actuals'][f'brier_{str(threshold)}'] = model['seq_data'][s]['df_w_actuals'].apply(lambda row: brier_score(row[predcol], row[outcome], threshold), axis=1)
        # Mean scores over all months and countries
        model['seq_data'][s]['mean_crps'] = model['seq_data'][s]['df_w_actuals']['crps'].mean()
        model['seq_data'][s]['mean_crps_onset'] = model['seq_data'][s]['df_w_actuals']['crps_onset'].mean()
        model['seq_data'][s]['mean_mse'] = model['seq_data'][s]['df_w_actuals']['mse'].mean()
        for b in bands:
            model['seq_data'][s][f'mean_coverage_{str(b)}'] = model['seq_data'][s]['df_w_actuals'][f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_coverage_nonzeros_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].loc[model['seq_data'][s]['df_w_actuals'][outcome] > 0, f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_coverage_zeros_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].loc[model['seq_data'][s]['df_w_actuals'][outcome] == 0, f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_coverage_bin_1_10_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].loc[(model['seq_data'][s]['df_w_actuals'][outcome] > 0) & (model['seq_data'][s]['df_w_actuals'][outcome] <= 10), f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_coverage_bin_11_100_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].loc[(model['seq_data'][s]['df_w_actuals'][outcome] > 10) & (model['seq_data'][s]['df_w_actuals'][outcome] <= 100), f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_coverage_bin_over100_{str(b)}'] = model['seq_data'][s]['df_w_actuals'].loc[model['seq_data'][s]['df_w_actuals'][outcome] > 100, f'coverage_{str(b)}'].mean()
            model['seq_data'][s][f'mean_winkler_{str(b)}'] = model['seq_data'][s]['df_w_actuals'][f'winkler_{str(b)}'].mean()
        for threshold in [1, 10, 100, 1000, 10000]:
            model['seq_data'][s][f'mean_brier_{str(threshold)}'] = model['seq_data'][s]['df_w_actuals'][f'brier_{str(threshold)}'].mean()
        print('  mean_crps:', model['seq_data'][s]['mean_crps'])
        # Mean scores by country:
        scores = [outcome,'crps', 'mse', 'coverage_80', 'coverage_90', 'coverage_95', 'coverage_98', 'winkler_80', 'winkler_90', 'winkler_95', 'winkler_98', 'brier_1', 'brier_10', 'brier_100', 'brier_1000', 'brier_10000','mean_pred']
        model['seq_data'][s]['country_means'] = model['seq_data'][s]['df_w_actuals'][scores].groupby('country_id').mean()
    # Mean scores over all sequences
    model['mean_crps'] = np.mean([model['seq_data'][s]['mean_crps'] for s in range(number_of_sequences)])
    model['mean_crps_onset'] = np.mean([model['seq_data'][s]['mean_crps_onset'] for s in range(number_of_sequences)])
    model['mean_mse'] = np.mean([model['seq_data'][s]['mean_mse'] for s in range(number_of_sequences)])
    for b in bands:
        model[f'mean_coverage_{str(b)}'] = np.mean([model['seq_data'][s][f'mean_coverage_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_coverage_nonzeros_{str(b)}']    = np.mean([model['seq_data'][s][f'mean_coverage_nonzeros_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_coverage_zeros_{str(b)}']       = np.mean([model['seq_data'][s][f'mean_coverage_zeros_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_coverage_bin_1_10_{str(b)}']     = np.mean([model['seq_data'][s][f'mean_coverage_bin_1_10_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_coverage_bin_11_100_{str(b)}']   = np.mean([model['seq_data'][s][f'mean_coverage_bin_11_100_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_coverage_bin_over100_{str(b)}'] = np.mean([model['seq_data'][s][f'mean_coverage_bin_over100_{str(b)}'] for s in range(number_of_sequences)])
        model[f'mean_winkler_{str(b)}'] = np.mean([model['seq_data'][s][f'mean_winkler_{str(b)}'] for s in range(number_of_sequences)])
    for threshold in [1, 10, 100, 1000, 10000]:
        model[f'mean_brier_{str(threshold)}'] = np.mean([model['seq_data'][s][f'mean_brier_{str(threshold)}'] for s in range(number_of_sequences)])
    model['mean_pred'] = np.mean([model['seq_data'][s]['df_w_actuals']['mean_pred'].mean() for s in range(number_of_sequences)])
    # Mean scores when dropping Ethiopia (57) and Ukraine (117)
    model['mean_crps_wo_ethukr'] = np.mean([model['seq_data'][s]['df_w_actuals']['crps'].drop(index=57, level=1).drop(index=117, level=1).mean() for s in range(number_of_sequences)])
    
    # Mean scores overall
    print('Overall mean_crps:', model['mean_crps'])
            


wild_rose
Sequence: 0
  mean_crps: 83.53941241884817
Overall mean_crps: 83.53941241884817
wuthering_heights
Sequence: 0
  mean_crps: 86.40488708900524
Overall mean_crps: 86.40488708900524
lovely_creature
Sequence: 0
  mean_crps: 86.33865037812681
Overall mean_crps: 86.33865037812681
purple_haze
Sequence: 0
  mean_crps: 86.086314849331
Overall mean_crps: 86.086314849331
great_gig
Sequence: 0
  mean_crps: 87.14274669808027
Overall mean_crps: 87.14274669808027
fourtieth_symphony
Sequence: 0
  mean_crps: 85.89829678708551
Overall mean_crps: 85.89829678708551
ensemble_raw
Sequence: 0
  mean_crps: 82.60751066579328
Overall mean_crps: 82.60751066579328
ensemble_smoothed
Sequence: 0
  mean_crps: 81.8933808452429
Overall mean_crps: 81.8933808452429
ensemble_smoothed_no_wild_rose
Sequence: 0
  mean_crps: 82.00275902322088
Overall mean_crps: 82.00275902322088
ensemble_smoothed_no_wuthering_heights
Sequence: 0
  mean_crps: 81.97858229848697
Overall mean_crps: 81.97858229848697
ensemble_smoothed_no

In [52]:
model


{'name': 'wild_rose',
 'stub': 'predictions_validation_20251008_225829_',
 'weight': 500,
 'seq_data': [{'first_month': 493,
   'last_month': 528,
   'samples': 500,
   'data':                                                        pred_lr_sb_best  \
   month_id country_id                                                      
   493      1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            2           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            3           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            4           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            5           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
   ...                                                                ...   
   528      242         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            243         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
            244         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [60]:
# Identify countries with highest mean crps for the ensemble model

for model in model_list:
    print(model['name'])
    print('Overall mean_crps:')
    print(model['seq_data'][s]['country_means'][['lr_sb_best','crps','winkler_90']].sort_values(ascending=False, by='crps').head(5))


wild_rose
Overall mean_crps:
             lr_sb_best         crps     winkler_90
country_id                                         
57          8040.027778  8007.707275  159471.943056
117         4514.027778  4459.447807   88386.788889
133         1026.194444   981.086227   14060.069444
124          752.694444   644.550052   11317.904167
218          641.055556   641.055556   12821.111111
wuthering_heights
Overall mean_crps:
             lr_sb_best         crps     winkler_90
country_id                                         
57          8040.027778  7897.510693  155594.236111
117         4514.027778  4512.211062   90137.052778
133         1026.194444  1047.466492   13495.363889
124          752.694444   659.831450   11561.276389
218          641.055556   641.055556   12821.111111
lovely_creature
Overall mean_crps:
             lr_sb_best         crps     winkler_90
country_id                                         
57          8040.027778  8107.348889  160730.522222
117         451

In [ ]:
df = model['seq_data'][s]['df_w_actuals'].copy()
print(df.tail(10))
df.drop(246, level=1, inplace=True)
print(df.tail(10))


In [ ]:
df = model['seq_data'][s]['df_w_actuals']
df = df[df[outcome]>0]
cols = ['country_name', outcome,'mean_pred','fatality_categories','categories_mode','crps','lower_90','higher_90','coverage_90']
df[cols].tail(20)


In [61]:
# Prepping for evaluation of aggregated predictions
# Aggregate predictions and actuals over months for each country

def sum_samples_country(country,cnt_df,predcol,samples=2500):
    dft = df.apply(lambda x: np.array(x))
    try:    
        npa = np.stack(dft.to_numpy())
        npa.shape
        result = list(npa.sum(axis=0))
    except:
        # If the arrays are of different lengths, we need to trim them
        for i in dft.index:
            dft[i] = dft[i][0:samples]
        result = np.stack(dft.to_numpy())
    return result



def aggregate_over_months(df, predcol, outcome):
    ''' Aggregate predictions and actuals over months for each country'''
    min_val = df.index.min()
    print('Number of samples per country month:', len(df[predcol].loc[min_val]))
    df_aggregated = df[[outcome]].groupby(['country_id']).sum().copy()
    df.reset_index(inplace=True)
#    df = df[df['country_id']>=240].copy() # Exclude countries with id < 240 (non-countries)
#   print(df.head(5))
    for cnt, row in df_aggregated.iterrows():
        df_cnt = df[df['country_id']==cnt].copy()
        if cnt > 242:
            print('df_cnt', cnt, len(df_cnt), df_cnt.head(3))
        row['pred_lr_sb_best'] = sum_samples_country(cnt, df_cnt[predcol],predcol)
    
    return df_aggregated

#df = model['seq_data'][s]['data'].merge(actuals_df, left_index=True, right_on=['month_id', 'country_id'], how='left')
#df = df[[predcol, outcome]].copy()
#print(df.head(5))
#df_aggregated = aggregate_over_months(df, predcol, outcome)
#df_aggregated.head()

In [62]:
stepæø


NameError: name 'stepæø' is not defined

## Presenting results

In [63]:
def print_scores(name, context, mean_pred, mean_crps, mean_crps_onset, mean_crps_wo_ethukr, mean_mse, mean_coverage_80, mean_coverage_90, mean_coverage_95, mean_coverage_98, mean_coverage_nonzeros_80, mean_coverage_nonzeros_90, mean_coverage_nonzeros_95, mean_coverage_nonzeros_98, mean_coverage_zeros_90, mean_coverage_bin_1_10_90, mean_coverage_bin_11_100_90, mean_coverage_bin_over100_90, mean_winkler_80, mean_winkler_90, mean_winkler_95, mean_winkler_98, mean_brier_1, mean_brier_10, mean_brier_100, mean_brier_1000, mean_brier_10000):
    print(f"Model: {name}, {context}")
    print(f"CRPS={mean_crps:.2f}, CRPS onset={mean_crps_onset:.4f}, CRPS without Ethiopia and Ukraine: {mean_crps_wo_ethukr:.2f}, MSE={mean_mse:.0f}, Mean prediction={mean_pred:.2f}")
    print(f"Coverage 80%={mean_coverage_80:.3f},  90%={mean_coverage_90:.3f}, 95%={mean_coverage_95:.3f}, 98%={mean_coverage_98:.3f}")
    print(f"Coverage non-zeros 80%={mean_coverage_nonzeros_80:.3f},  90%={mean_coverage_nonzeros_90:.3f}, 95%={mean_coverage_nonzeros_95:.3f}, 98%={mean_coverage_nonzeros_98:.3f}")
    print(f'Binned 90% coverage zeros={mean_coverage_zeros_90:.3f}, (1-10)={mean_coverage_bin_1_10_90:.3f}, (11-100)={mean_coverage_bin_11_100_90:.3f}, (>100)={mean_coverage_bin_over100_90:.3f}   ')
    print(f"Winkler 80%={mean_winkler_80:.1f}, 90%={mean_winkler_90:.1f}, 95%={mean_winkler_95:.1f}, 98%={mean_winkler_98:.1f}")
    print(f"Brier 1={mean_brier_1:.3f}, 10={mean_brier_10:.3f}, 100={mean_brier_100:.3f}, 1000={mean_brier_1000:.4f}, 10000={mean_brier_10000:.5f}")
    print(80*'-')
    
print(80*'-')
for model in model_list:
    print_scores(model['name'], 'All country months and sequences', model['mean_pred'], model['mean_crps'], model['mean_crps_onset'], model['mean_crps_wo_ethukr'], model['mean_mse'], model['mean_coverage_80'], model['mean_coverage_90'], model['mean_coverage_95'], model['mean_coverage_98'],  model['mean_coverage_nonzeros_80'], model['mean_coverage_nonzeros_90'], model['mean_coverage_nonzeros_95'], model['mean_coverage_nonzeros_98'], model['mean_coverage_zeros_90'], model['mean_coverage_bin_1_10_90'], model['mean_coverage_bin_11_100_90'], model['mean_coverage_bin_over100_90'], model['mean_winkler_80'], model['mean_winkler_90'], model['mean_winkler_95'], model['mean_winkler_98'], model['mean_brier_1'], model['mean_brier_10'], model['mean_brier_100'], model['mean_brier_1000'], model['mean_brier_10000'])


--------------------------------------------------------------------------------
Model: wild_rose, All country months and sequences
CRPS=83.54, CRPS onset=0.0492, CRPS without Ethiopia and Ukraine: 18.46, MSE=4537154, Mean prediction=11.40
Coverage 80%=0.875,  90%=0.884, 95%=0.890, 98%=0.895
Coverage non-zeros 80%=0.259,  90%=0.313, 95%=0.350, 98%=0.382
Binned 90% coverage zeros=0.994, (1-10)=0.011, (11-100)=0.526, (>100)=0.311   
Winkler 80%=815.7, 90%=1595.1, 95%=3129.7, 98%=7676.2
Brier 1=0.077, 10=0.048, 100=0.032, 1000=0.0069, 10000=0.00160
--------------------------------------------------------------------------------
Model: wuthering_heights, All country months and sequences
CRPS=86.40, CRPS onset=0.0492, CRPS without Ethiopia and Ukraine: 21.66, MSE=4593364, Mean prediction=17.68
Coverage 80%=0.878,  90%=0.886, 95%=0.891, 98%=0.895
Coverage non-zeros 80%=0.292,  90%=0.342, 95%=0.374, 98%=0.400
Binned 90% coverage zeros=0.990, (1-10)=0.006, (11-100)=0.546, (>100)=0.402   
Winkl

In [44]:
model['mean_coverage_bin1_10_90']

KeyError: 'mean_coverage_bin1_10_90'

In [ ]:
actuals_df.loc[493].loc[70]

In [ ]:
for country in ['Ethiopia','Sudan','Mali','Chad','Ukraine','Afghanistan','Israel','Yemen','Syria','Somalia','Burkina Faso','Nigeria', 'Central African Republic']:
    print(actuals_df.loc[actuals_df['country_name']==country].loc[493])


In [ ]:
df_i = model['seq_data'][s]['country_means']
df_i.loc[df_i['crps']>=10]

In [ ]:
# Country-wise scores
countries_to_evaluate = [(57,'Ethiopia')]
countries_to_evaluate = [(57, 'Ethiopia'),(245, 'Sudan'),(50, 'Mali'),(214,'Chad'),(117,'Ukraine'),(133,'Afghanistan'),(218,'Israel'),(124,'Yemen'),(220,'Syria'),(120,'Somalia'),(47,'Burkina Faso'),(79,'Nigeria'), (70,'Central African Republic')]
for country in countries_to_evaluate:
    obs = int(model_list[0]['seq_data'][s]['country_means']['lr_sb_best'].loc[country[0]])
    print(country[1], ', observed fatalities:', obs)
    for model in model_list[:]:
        df = model['seq_data'][s]['country_means'].loc[country[0]]
        modelname = model['name']
        fill = ' '* (24-len(modelname))
        crps = df['crps']
        winkler = df['winkler_90']
        mean_pred = df['mean_pred']
        mean_obs = df['lr_sb_best']
        print(f'{modelname}{fill} Mean predicted: {mean_pred:.1f} CRPS: {crps:.1f}, Winkler 90% {winkler:.0f}')


In [ ]:
#Country line plots

import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import datetime

def month_id_to_date(month_id: int) -> str:
    """Converts a month_id to a string with the month and year.
    Parameters
    ----------
    month_id : int
        A count of months starting (from 1) on January 1980.
    Returns
    -------
    str
        The month and year of the month_id in "Month Year" format.
    """
    start_date = datetime.date(1980, 1, 1)
    target_date = start_date + relativedelta(months=month_id-1)
    return target_date.strftime("%B %Y")

def plot_with_bands(sequence, model, country_name, bands, savefig=False):
    df = model_list[model]['seq_data'][sequence]['df_w_actuals'].copy()
    df.reset_index(inplace=True)
    df_country = df.loc[df['country_name']==country_name] 
    plt.figure(figsize=(12, 6))
    alpha = 0.5
    plt.bar(df_country['month_id'],df_country[outcome],label='Mean Prediction', color = 'lightblue', linestyle='solid')
    plt.plot(df_country['month_id'],df_country['mean_pred'],label='Mean Prediction', color = 'black', linestyle='solid')
    #colors = ['purple', 'blueviolet', 'blue', 'teal', 'green','yellowgreen','yellow','gold','orange','darkorange','red','purple']
    colors = ['purple', 'green','yellow','red']
    for band in bands:
        colname = f'lower_{str(band)}'
        plt.plot(df_country['month_id'],df_country[colname],label=colname, color = colors[bands.index(band)], linestyle='dashed')
        colname = f'higher_{str(band)}'
        plt.plot(df_country['month_id'],df_country[colname],label=colname, color = colors[bands.index(band)], linestyle='dashed')
    #plt.bar(df['month_id'],df['sb_best'],alpha=alpha)
    first_plot_month = df_country['month_id'].min()
    last_plot_month = df_country['month_id'].max()
    plt.title(f'Actuals versus predictions (mean and prediction bands), {country_name}, model: {model_list[model]["name"]}, sequence: {sequence} \n First month: {first_plot_month} {month_id_to_date(first_plot_month)}, last month: {last_plot_month} {month_id_to_date(last_plot_month)} \n Country-sequence mean metrics: CRPS {df_country["crps"].mean():.2f}, CRPS onset {df_country["crps_onset"].mean():.2f}, Winkler 90% {df_country["winkler_90"].mean():.1f}, Coverage 90% {df_country["coverage_90"].mean():.3f}, MSE {df_country["mse"].mean():.0f}')
    plt.xlabel('Month ID')
    plt.ylabel('SB Best')
    plt.legend()
    if savefig:
        plt.savefig(f'plots/predictions_{country_name}_model_{model_list[model]["name"]}_sequence_{sequence}.png')
        plt.close()
    else:
        plt.show()

for country_to_plot in ['Burkina Faso','Mozambique','Nigeria','Sudan', 'South Sudan', 'Kenya', 'Ethiopia','Somalia','Yemen','Malawi', 'Israel']:
#for country_to_plot in ['Burkina Faso']:
    sequence_list = [0,number_of_sequences-1]
    for model in range(len(model_list)):
    #for model in [0,1,2,3,4,6,7,8,9]:
        for sequence in sequence_list:
            plot_with_bands(sequence, model, country_to_plot, [80, 90, 95, 98],savefig=True)


In [ ]:
month_id_to_date(model_list[model]['seq_data'][sequence]['df_w_actuals'].first_valid_index()[0])

In [ ]:
#df = model_list[0]['seq_data'][0]['df_w_actuals'].copy()

def extract_category(df):
    fatalities_df = df['fatality_categories']
    df_expanded = pd.DataFrame(fatalities_df.tolist(), index=fatalities_df.index)
    df_expanded.columns = ['0-1', '1-10', '10-100', '100-1000', '1000-10000']
    df_expanded = pd.concat([df, df_expanded], axis=1)
    return df_expanded
#df_expanded = extract_category(df)
#print(df.tail(10))
#print(df_expanded.tail(10))
def plot_categories(sequence, model, country_name, thresholds, stacked=False):
    
    df = model_list[model]['seq_data'][sequence]['df_w_actuals'].copy()
    df = extract_category(df)
    df.reset_index(inplace=True)
    df_country = df.loc[df['country_name']==country_name] 
#    plt.figure(figsize=(12, 6))
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    alpha = 0.5
    ax1.bar(df_country['month_id'],df_country['sb_best'],label='Actual fatalities', color = 'gray', linestyle='solid')
    ax1.set_ylabel('SB Best')
    ax1.legend()

    colors = ['purple', 'blueviolet', 'blue', 'teal', 'green','yellowgreen','yellow','gold','orange','darkorange','red','purple']
    if stacked:
        ax2.stackplot(df_country['month_id'], df_country[thresholds].T, labels=thresholds, colors=colors[0:len(thresholds)], alpha=0.5)
        ax2.set_ylabel('Proportion in category (stacked)')
    else:
        for t in thresholds:
            ax2.plot(df_country['month_id'],df_country[t],label=colname, color = colors[thresholds.index(t)], linestyle='dashed')
        ax2.set_ylabel('Proportion in category')
    ax2.legend(thresholds, title='Fatality categories', loc='upper right')
    plt.title(f'Actuals versus predictions (mean and prediction bands), {country_name}, model: {model_list[model]["name"]}, sequence: {sequence}')
    plt.xlabel('Month ID')
    plt.show()
    
country_to_plot = 'Sudan'
for model in range(len(model_list)):
    plot_categories(0, model, country_to_plot, ['0-1', '1-10','10-100', '100-1000', '1000-10000'], stacked=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
x = np.arange(1, 6)
y1 = np.array([1, 5, 2, 4, 3])
y2 = np.array([2, 3, 4, 1, 5])
y3 = np.array([3, 2, 1, 5, 4])

# Create the stacked area chart
plt.stackplot(x, y1, y2, y3, labels=['Series 1', 'Series 2', 'Series 3'])

# Add labels and title
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Stacked Area Chart Example')
plt.legend(loc='upper left')

# Display the chart
plt.show()

In [ ]:

s = 0
plt.figure(figsize=(12, 6))
country_id = 246  # Example country_id
df = model_list[-1]['seq_data'][s]['df_w_actuals'].copy()
df.reset_index(inplace=True)
df_country = df.loc[df['country_name']=='Sudan'] 
#print(df_country.head())
alpha = 0.5
plt.bar(df_country['month_id'],df_country['sb_best'],label='Mean Prediction', color = 'gray', linestyle='solid')
plt.plot(df_country['month_id'],df_country['mean_pred'],label='Mean Prediction', color = 'gray', linestyle='solid')
colors = ['purple','blueviolet','blue','teal', 'green','yellowgreen','yellow','gold','orange','darkorange','red','purple']
for band in bands:
    colname = f'lower_{str(band)}'
    plt.plot(df_country['month_id'],df_country[colname],label=colname, color = colors[bands.index(band)], linestyle='dashed')
    colname = f'higher_{str(band)}'
    plt.plot(df_country['month_id'],df_country[colname],label=colname, color = colors[bands.index(band)], linestyle='dashed')
#plt.bar(df['month_id'],df['sb_best'],alpha=alpha)
plt.title(f'Actuals versus predictions (mean and prediction bands), {country_name}')
plt.xlabel('Month ID')
plt.ylabel('SB Best')
plt.legend()
plt.show()


In [ ]:
model_list[-1]['seq_data'][0]['df_w_actuals']

In [ ]:
df.loc[(slice(None), 246), :] 